In [93]:
import json
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [123]:
file_1 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/raw_prices_carlsjr_non_ca_05162024.csv"
file_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/raw_prices_hardees_non_ca_05162024.csv"

carls = pd.read_csv(file_1)
hardees = pd.read_csv(file_2)

file_path_2 = "/Users/alyssanguyen/Desktop/IRLE_scraping/csv_files/uszips.csv"
ca_zip_count = pd.read_csv(file_path_2)

carls['restaurant_name']  = 'Carls Jr'
hardees['restaurant_name']  = 'Hardees'


ca_ff = pd.concat([carls, hardees])

In [124]:
ca_ff

,menu_item,menu_item_price,menu_item_calories,restaurant_address,restaurant_name
0,Big Angus El Diablo Combo,3.19,270,"1700 W State St, Boise, ID, 83702, US",Carls Jr
1,Double Take – Pick Two w/Free Drink,5.49,670,"1700 W State St, Boise, ID, 83702, US",Carls Jr
2,Double Famous Star®,6.99,920,"1700 W State St, Boise, ID, 83702, US",Carls Jr
3,Big Angus Famous Star®,7.99,830,"1700 W State St, Boise, ID, 83702, US",Carls Jr
4,Single Western Bacon Cheeseburger®,5.99,760,"1700 W State St, Boise, ID, 83702, US",Carls Jr
...,...,...,...,...,...
1252,Big Cheeseburger,5.69,540,"2450 E Layton Ave, Saint Francis, WI, 53235-60...",Hardees
1253,Double Big Cheeseburger,7.69,760,"2450 E Layton Ave, Saint Francis, WI, 53235-60...",Hardees
1254,Monster,9.79,1160,"2450 E Layton Ave, Saint Francis, WI, 53235-60...",Hardees
1255,Bacon Cheeseburger,7.69,740,"2450 E Layton Ave, Saint Francis, WI, 53235-60...",Hardees


In [125]:
ca_zip_count = ca_zip_count[['zip', 'county_name']]

In [126]:
def price_list(x):
    return list(x)
def mean_non_zero(x):
    return np.mean(x[x != 0]) if np.any(x != 0) else 0

def median_non_zero(x):
    return np.median(x[x != 0]) if np.any(x != 0) else 0

def std_non_zero(x):
    return np.std(x[x != 0]) if np.any(x != 0) else 0

In [127]:
#Drop all the columns we don't need 
ca_ff_ = ca_ff.drop(columns=['menu_item_calories'])

#Drop all restaurants with no restaurant address 
#ca_ff_ = ca_ff_.dropna(subset=['restaurant_location'])

#Run if doing restaurant-specific processing i.e. Wendy's Hardee's etc 

ca_ff_ = ca_ff_.rename(columns = {'restaurant_address': "restaurant_location"})
#ca_ff_['restaurant_name'] = 'Carls Jr'
ca_ff_['restaurant_rating'] = np.nan
ca_ff_['number_of_ratings'] = np.nan

In [128]:
# Define a function to split prices by "-" or "/"
def split_and_average(price_str):
    # Split the price string by "-" or "/"
    prices = re.split(r'[-/]', price_str)
    # Convert prices to float and calculate the average
    prices = [float(price) for price in prices]
    return sum(prices) / len(prices)

# Apply the split_and_average function to each row in the menu_item_price column
ca_ff_['menu_item_price'] = ca_ff_['menu_item_price'].apply(split_and_average)

TypeError: expected string or bytes-like object

In [129]:
#cleaning up string columns 

ca_ff_['menu_item'] = ca_ff_['menu_item'].str.lower()
ca_ff_['restaurant_location'] = ca_ff_['restaurant_location'].str.lower()
ca_ff_['restaurant_name'] = ca_ff_['restaurant_name'].str.replace('_', ' ')

#remove special characters
ca_ff_['menu_item'] = ca_ff_['menu_item'].apply(lambda x: ''.join(ch for ch in x if ch.isalnum() or ch.isspace()))
ca_ff_

,menu_item,menu_item_price,restaurant_location,restaurant_name,restaurant_rating,number_of_ratings
0,big angus el diablo combo,3.19,"1700 w state st, boise, id, 83702, us",Carls Jr,NaN,NaN
1,double take pick two wfree drink,5.49,"1700 w state st, boise, id, 83702, us",Carls Jr,NaN,NaN
2,double famous star,6.99,"1700 w state st, boise, id, 83702, us",Carls Jr,NaN,NaN
3,big angus famous star,7.99,"1700 w state st, boise, id, 83702, us",Carls Jr,NaN,NaN
4,single western bacon cheeseburger,5.99,"1700 w state st, boise, id, 83702, us",Carls Jr,NaN,NaN
...,...,...,...,...,...,...
1252,big cheeseburger,5.69,"2450 e layton ave, saint francis, wi, 53235-60...",Hardees,NaN,NaN
1253,double big cheeseburger,7.69,"2450 e layton ave, saint francis, wi, 53235-60...",Hardees,NaN,NaN
1254,monster,9.79,"2450 e layton ave, saint francis, wi, 53235-60...",Hardees,NaN,NaN
1255,bacon cheeseburger,7.69,"2450 e layton ave, saint francis, wi, 53235-60...",Hardees,NaN,NaN


In [ ]:
#Filter to just Wendy's
ca_ff_wendy = ca_ff_[ca_ff_['restaurant_name'] == 'Wendy']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_wendy = ca_ff_wendy.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_wendy.columns = [' '.join(col).strip() for col in grouped_wendy.columns.values]

# #Second part of grouping 
wendy_lst = ['daves single', 'french fries', 'jr cheeseburger', 'jr hamburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_wendy = ca_ff_wendy[ca_ff_wendy['menu_item'].isin(wendy_lst)].sort_values('menu_item')
menu_items_wendy = menu_items_wendy.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_wendy_2 = menu_items_wendy.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_wendy_2[['specialty_item', 'fries', 'cheeseburger', 'hamburger']] = grouped_wendy_2['menu_item_price'].apply(pd.Series)
grouped_wendy_2.drop(columns=['menu_item_price'], inplace=True)

merged_wendy = pd.merge(grouped_wendy, grouped_wendy_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_wendy['combo'] = np.nan
merged_wendy

In [31]:
#Filter to just Burger King 
ca_ff_bk = ca_ff_[ca_ff_['restaurant_name'] == 'Burger King']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_bk = ca_ff_bk.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_bk.columns = [' '.join(col).strip() for col in grouped_bk.columns.values]

# #Second part of grouping 
#they don't have a plain hamburger FOR NOW using whopper jr 
bk_lst = ['cheeseburger', 'french fries', 'whopper', 'whopper jr']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_bk = ca_ff_bk[ca_ff_bk['menu_item'].isin(bk_lst)].sort_values('menu_item')
menu_items_bk = menu_items_bk.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_bk_2 = menu_items_bk.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_bk_2[['cheeseburger', 'fries', 'specialty_item', 'hamburger']] = grouped_bk_2['menu_item_price'].apply(pd.Series)
grouped_bk_2.drop(columns=['menu_item_price'], inplace=True)

merged_bk = pd.merge(grouped_bk, grouped_bk_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_bk['combo'] = np.nan
merged_bk

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,fries,specialty_item,hamburger,combo
0,Burger King,"100 river oaks cove, georgetown, tx, 78626, us",5.239020,3.99,3.170722,NaN,51,NaN,2.19,2.69,5.89,2.89,NaN
1,Burger King,"1001 east ben white boulevard, austin, tx, 787...",5.239020,3.99,3.170722,NaN,51,NaN,2.19,2.69,5.89,2.89,NaN
2,Burger King,"1004 north memorial parkway, huntsville, al, 3...",4.044839,3.39,1.944195,NaN,31,NaN,2.29,2.89,5.89,3.09,NaN
3,Burger King,"10205 e. freeway, houston, tx, 77013, us",5.052745,3.99,3.071115,NaN,51,NaN,1.89,2.49,5.49,2.59,NaN
4,Burger King,"1027 union ave, memphis, tn, 38104, us",5.023333,3.99,2.915677,NaN,57,NaN,2.19,3.09,5.59,3.39,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,Burger King,"819 memorial blvd, murfreesboro, tn, 37129, us",4.234118,3.29,2.558999,NaN,119,NaN,2.19,3.29,5.69,3.59,NaN
167,Burger King,"819 south 3rd street, wilmington, nc, 28401, us",5.049649,4.19,2.936513,NaN,57,NaN,2.19,3.09,5.69,3.59,NaN
168,Burger King,"8378 w overland rd., boise, id, 83709, us",4.227500,3.29,2.339660,NaN,24,NaN,2.29,3.29,5.79,3.29,NaN
169,Burger King,"85 tunnel road, asheville, nc, 28805, us",4.953158,3.89,2.889341,NaN,57,NaN,1.99,3.09,5.39,3.09,NaN


In [130]:
#Filter to Carl's Jr 

ca_ff_carls = ca_ff_[ca_ff_['restaurant_name'] == 'Carls Jr']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_carls = ca_ff_carls.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_carls.columns = [' '.join(col).strip() for col in grouped_carls.columns.values]


#Second part of grouping 
carls_lst = ['california classic double cheeseburger', 'single big carl']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_carls = ca_ff_carls[ca_ff_carls['menu_item'].isin(carls_lst)].sort_values('menu_item')
menu_items_carls = menu_items_carls.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_carls_2 = menu_items_carls.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_carls_2[['cheeseburger', 'specialty_item']] = grouped_carls_2['menu_item_price'].apply(pd.Series)
grouped_carls_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_carls = pd.merge(grouped_carls, grouped_carls_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_carls['fries'] = np.nan
merged_carls['combo'] = np.nan
merged_carls['hamburger'] = np.nan

merged_carls

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,cheeseburger,specialty_item,fries,combo,hamburger
0,Carls Jr,"1039 w university ave, georgetown, tx, 78628, us",6.940000,7.04,1.972942,NaN,18,NaN,3.49,5.39,NaN,NaN,NaN
1,Carls Jr,"1155 w riverdale rd unit a riverdale, ut 84405",6.617778,6.99,1.497704,NaN,18,NaN,3.99,5.99,NaN,NaN,NaN
2,Carls Jr,"1700 w state st, boise, id, 83702, us",7.056667,7.49,1.941649,NaN,18,NaN,3.99,4.99,NaN,NaN,NaN
3,Carls Jr,"2118 w 1700 s syracuse, ut 84075, us",6.617778,6.99,1.497704,NaN,18,NaN,3.99,5.99,NaN,NaN,NaN
4,Carls Jr,"493 n milwaukee st, boise, id, 83704, us",7.056667,7.49,1.941649,NaN,18,NaN,3.99,4.99,NaN,NaN,NaN
5,Carls Jr,"4994 e 41st, tulsa, ok, 74135, us",6.795789,7.19,1.640819,NaN,19,NaN,4.09,5.99,NaN,NaN,NaN
6,Carls Jr,"5722 s 49th west ave tulsa, ok 74107",6.795789,7.19,1.640819,NaN,19,NaN,4.09,5.99,NaN,NaN,NaN
7,Carls Jr,"582 n main street clearfield, ut 84015, us",6.617778,6.99,1.497704,NaN,18,NaN,3.99,5.99,NaN,NaN,NaN
8,Carls Jr,"925 w antelope drive, layton, ut, 84041, us",6.617778,6.99,1.497704,NaN,18,NaN,3.99,5.99,NaN,NaN,NaN


In [131]:
ca_ff_hardee = ca_ff_[ca_ff_['restaurant_name'] == 'Hardees']

#First part of grouping 

agg_funcs = {
    'menu_item_price': [mean_non_zero, median_non_zero, std_non_zero],  # calculate the average, median, and standard dev PRICE
    'restaurant_rating': 'mean', # calculate the average RATING 
    'menu_item' : 'count',
    'number_of_ratings': 'first'
}

grouped_hardee = ca_ff_hardee.groupby(['restaurant_name','restaurant_location']).agg(agg_funcs).reset_index()
grouped_hardee.columns = [' '.join(col).strip() for col in grouped_hardee.columns.values]


#Second part of grouping 
carls_lst = ['famous star','small cheeseburger', 'small hamburger']

# Filter rows where 'menu_item' contains any item in mcd_lst
menu_items_hardee = ca_ff_hardee[ca_ff_hardee['menu_item'].isin(carls_lst)].sort_values('menu_item')
menu_items_hardee = menu_items_hardee.drop_duplicates(subset=['restaurant_name', 'restaurant_location', 'menu_item'])

grouped_hardee_2 = menu_items_hardee.groupby(['restaurant_name', 'restaurant_location'])['menu_item_price'].agg(price_list).reset_index()

grouped_hardee_2[['specialty_item', 'cheeseburger', 'hamburger']] = grouped_hardee_2['menu_item_price'].apply(pd.Series)
grouped_hardee_2.drop(columns=['menu_item_price'], inplace=True)

#Merging the grouped dfs together 
merged_hardee = pd.merge(grouped_hardee, grouped_hardee_2, on=['restaurant_name', 'restaurant_location'], how='inner')
merged_hardee['fries'] = np.nan
merged_hardee['combo'] = np.nan
merged_hardee

,restaurant_name,restaurant_location,menu_item_price mean_non_zero,menu_item_price median_non_zero,menu_item_price std_non_zero,restaurant_rating mean,menu_item count,number_of_ratings first,specialty_item,cheeseburger,hamburger,fries,combo
0,Hardees,"1 gateway blvd s, savannah, ga, 31419-7551, us",7.443846,7.09,1.345428,NaN,13,NaN,6.39,NaN,NaN,NaN,NaN
1,Hardees,"10 e clemmonsville rd, winston salem, nc, 2712...",7.228462,6.89,1.439305,NaN,13,NaN,6.29,NaN,NaN,NaN,NaN
2,Hardees,"10 hwy 17 n, surfside beach, sc, 29575, us",7.228462,6.89,1.439305,NaN,13,NaN,6.29,NaN,NaN,NaN,NaN
3,Hardees,"100 vandora springs rd., garner, nc, 27529, us",6.016667,5.99,1.774060,NaN,15,NaN,5.39,2.69,2.39,NaN,NaN
4,Hardees,"1000 acorn dr, nashville, tn, 37210, us",6.873333,6.64,1.170351,NaN,12,NaN,6.09,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,Hardees,"9020 east 21st street, indianapolis, in, 46229...",7.420769,7.09,1.367977,NaN,13,NaN,6.39,NaN,NaN,NaN,NaN
91,Hardees,"915 army post rd, des moines, ia, 50315, us",7.420769,7.49,1.536422,NaN,13,NaN,5.99,NaN,NaN,NaN,NaN
92,Hardees,"917 allison-bonnett memorial dr, hueytown, al,...",7.066923,6.89,1.358253,NaN,13,NaN,5.99,NaN,NaN,NaN,NaN
93,Hardees,"940 thornton rd., lithia springs, ga, 30122, us",7.451538,7.19,1.377031,NaN,13,NaN,5.89,NaN,NaN,NaN,NaN


In [132]:
stacked = pd.concat([merged_carls, merged_hardee])

In [133]:
pattern = r",\s*([a-zA-Z]{2})\s*,?\s*(\d{5}(?:-\d{4})?)"

def extract_state_zip(address):
    match = re.search(pattern, address)
    if match:
        state, zip_code = match.groups()
        return state, zip_code
    else:
        return None, None

# Apply the function to extract state and zip code
stacked[['state', 'zip']] = stacked['restaurant_location'].apply(lambda x: pd.Series(extract_state_zip(x)))
stacked['zip'] = stacked['zip'].str.split('-').str[0].astype(int)

#Get county 
stacked = stacked.merge(ca_zip_count, on = 'zip')

In [120]:
specific_date = datetime.strptime('05162024', '%m%d%Y')
# Assign the datetime object to the entire 'date' column
stacked['date'] = specific_date
stacked['uber_eats'] = 0
stacked['post_policy'] = 1
stacked['fast_food'] = 1 
stacked['local'] = 0 

In [134]:
stacked.to_csv("final_processed_prices_carlsjr_nonca_05162024.csv")